In [ ]:
# Deviates from the original installation instructions.
# https://issuemode.com/issues/tensorflow/serving/92945160
!wget 'http://storage.googleapis.com/tensorflow-serving-apt/pool/tensorflow-model-server-universal-2.8.0/t/tensorflow-model-server-universal/tensorflow-model-server-universal_2.8.0_all.deb'
!dpkg -i tensorflow-model-server-universal_2.8.0_all.deb

--2022-12-16 02:47:21--  http://storage.googleapis.com/tensorflow-serving-apt/pool/tensorflow-model-server-universal-2.8.0/t/tensorflow-model-server-universal/tensorflow-model-server-universal_2.8.0_all.deb
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.28.128, 74.125.134.128, 173.194.210.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.28.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 335421916 (320M) [application/x-debian-package]
Saving to: ‘tensorflow-model-server-universal_2.8.0_all.deb’

tensorflow-model-se 100%[===================>] 319.88M  52.0MB/s    in 7.1s    

2022-12-16 02:47:28 (45.1 MB/s) - ‘tensorflow-model-server-universal_2.8.0_all.deb’ saved [335421916/335421916]

Selecting previously unselected package tensorflow-model-server-universal.
(Reading database ... 124025 files and directories currently installed.)
Preparing to unpack tensorflow-model-server-universal_2.8.0_all.deb ...
Unpacki

In [ ]:
!apt install apt-transport-https curl gnupg
!curl -fsSL https://bazel.build/bazel-release.pub.gpg | gpg --dearmor > bazel.gpg
!mv bazel.gpg /etc/apt/trusted.gpg.d/
!echo "deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo tee /etc/apt/sources.list.d/bazel.list
!apt update && apt install bazel

In [ ]:
!git clone https://github.com/keras-team/keras-cv.git

In [ ]:
%cd keras-cv

In [ ]:
!git checkout tags/v0.3.5 -b dev

!python3 build_deps/configure.py

!bazel build build_pip_pkg
!bazel-bin/build_pip_pkg wheels

In [ ]:
!pip install /content/keras-cv/wheels/keras_cv-*.whl

In [ ]:
import time
import base64
import keras_cv
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
keras_cv.__version__

'0.3.4'

In [ ]:
from keras_cv.models.stable_diffusion.text_encoder import TextEncoder
from keras_cv.models.stable_diffusion.diffusion_model import DiffusionModel
from keras_cv.models.stable_diffusion.decoder import Decoder

In [ ]:
MAX_PROMPT_LENGTH = 77
IMG_HEIGHT = 512
IMG_WIDTH = 512
 
text_encoder = TextEncoder(MAX_PROMPT_LENGTH)
diffusion_model = DiffusionModel(IMG_HEIGHT, IMG_WIDTH, MAX_PROMPT_LENGTH)
decoder = Decoder(IMG_HEIGHT, IMG_WIDTH)

In [ ]:
text_encoder_weights_fpath = keras.utils.get_file(
    origin="https://huggingface.co/fchollet/stable-diffusion/resolve/main/kcv_encoder.h5",
    file_hash="4789e63e07c0e54d6a34a29b45ce81ece27060c499a709d556c7755b42bb0dc4",
)
diffusion_model_weights_fpath = keras.utils.get_file(
    origin="https://huggingface.co/fchollet/stable-diffusion/resolve/main/kcv_diffusion_model.h5",
    file_hash="8799ff9763de13d7f30a683d653018e114ed24a6a819667da4f5ee10f9e805fe",
)
decoder_weights_fpath = keras.utils.get_file(
    origin="https://huggingface.co/fchollet/stable-diffusion/resolve/main/kcv_decoder.h5",
    file_hash="ad350a65cc8bc4a80c8103367e039a3329b4231c2469a1093869a345f55b1962",
)

In [ ]:
text_encoder.load_weights(text_encoder_weights_fpath)
diffusion_model.load_weights(diffusion_model_weights_fpath)
decoder.load_weights(decoder_weights_fpath)

In [ ]:
import tensorflow as tf
from tensorflow import keras

from keras_cv.models.stable_diffusion.clip_tokenizer import SimpleTokenizer
tokenizer = SimpleTokenizer()

def _get_pos_ids():
    return tf.convert_to_tensor([list(range(MAX_PROMPT_LENGTH))], dtype=tf.int32)

def encode_text(prompt):
    """Encodes a prompt into a latent text encoding.
    The encoding produced by this method should be used as the
    `encoded_text` parameter of `StableDiffusion.generate_image`. Encoding
    text separately from generating an image can be used to arbitrarily
    modify the text encoding priot to image generation, e.g. for walking
    between two prompts.
    Args:
        prompt: a string to encode, must be 77 tokens or shorter.
    Example:
    ```python
    from keras_cv.models import StableDiffusion
    model = StableDiffusion(img_height=512, img_width=512, jit_compile=True)
    encoded_text  = model.encode_text("Tacos at dawn")
    img = model.generate_image(encoded_text)
    ```
    """
    # Tokenize prompt (i.e. starting context)
    inputs = tokenizer.encode(prompt)
    if len(inputs) > MAX_PROMPT_LENGTH:
        raise ValueError(
            f"Prompt is too long (should be <= {MAX_PROMPT_LENGTH} tokens)"
        )
    phrase = inputs + [49407] * (MAX_PROMPT_LENGTH - len(inputs))
    phrase = tf.convert_to_tensor([phrase], dtype=tf.int32)

    context = text_encoder.predict_on_batch([phrase, _get_pos_ids()])

    return context

encoded_text = encode_text("photograph of an astronaut riding a horse")

In [ ]:
inputs = tokenizer.encode("photograph of an astronaut riding a horse")
inputs

[49406, 8853, 539, 550, 18376, 6765, 320, 4558, 49407]

In [ ]:
phrase = inputs + [49407] * (MAX_PROMPT_LENGTH - len(inputs))
phrase = tf.convert_to_tensor([phrase], dtype=tf.int32)
phrase

<tf.Tensor: shape=(1, 77), dtype=int32, numpy=
array([[49406,  8853,   539,   550, 18376,  6765,   320,  4558, 49407,
        49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
        49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
        49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
        49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
        49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
        49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
        49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
        49407, 49407, 49407, 49407, 49407]], dtype=int32)>

In [ ]:
encoded_text

array([[[-0.3883766 ,  0.02294356, -0.05219662, ..., -0.48988292,
         -0.30660194,  0.06745391],
        [ 0.1156525 ,  0.4144913 ,  2.3049333 , ..., -1.7711414 ,
         -1.0141486 , -0.9208659 ],
        [-0.4335628 , -0.09044406,  0.722898  , ...,  0.8755375 ,
         -1.0948254 , -0.7246346 ],
        ...,
        [-3.080411  , -0.19198483, -0.33464575, ...,  0.5393776 ,
         -0.15574259,  0.60614014],
        [-3.0885706 , -0.24322483, -0.3512005 , ...,  0.5685484 ,
         -0.15004434,  0.5980401 ],
        [-3.0169518 , -0.24591564, -0.30276614, ...,  0.5613408 ,
         -0.10421933,  0.5855357 ]]], dtype=float32)

# Text Encoder in SavedModel

Assumes that tokens are prepared to be passed to the model with `SimpleTokenizer`

In [ ]:
text_encoder.inputs

[<KerasTensor: shape=(None, 77) dtype=int32 (created by layer 'tokens')>,
 <KerasTensor: shape=(None, 77) dtype=int32 (created by layer 'positions')>]

### 🛑 can't make concrete function, current codes simply calls the model itself without concrete function. need to investigate if this is an error

In [ ]:
# 
m_call = tf.function(text_encoder.call).get_concrete_function(
        tf.TensorSpec(
            shape=[1, 77], dtype=tf.int32, name='token'
        ),
        tf.TensorSpec(
            shape=[1, 77], dtype=tf.int32, name='positions'
        )
    )

AssertionError: ignored

In [ ]:
from keras_cv.models.stable_diffusion.constants import _UNCONDITIONAL_TOKENS

signature_dict = { "tokens": tf.TensorSpec(shape=[], dtype=tf.int32, name="tokens"),
                   "batch_size": tf.TensorSpec(shape=[], dtype=tf.int32, name="batch_size")}

def model_exporter(model: tf.keras.Model):
    MAX_PROMPT_LENGTH = 77
    pos_ids = tf.convert_to_tensor([list(range(MAX_PROMPT_LENGTH))], dtype=tf.int32)

    @tf.function(input_signature=[signature_dict])
    def serving_fn(string_input):
        batch_size = string_input["batch_size"]

        # context
        encoded_text = model([string_input['tokens'], pos_ids])
        encoded_text = tf.squeeze(encoded_text)

        if encoded_text.shape.rank == 2:
            encoded_text = tf.repeat(
                tf.expand_dims(encoded_text, axis=0), batch_size, axis=0
            )
        context = encoded_text

        # unconditional context
        unconditional_tokens = tf.convert_to_tensor([_UNCONDITIONAL_TOKENS], dtype=tf.int32)
        unconditional_context = model([unconditional_tokens, pos_ids])        

        unconditional_context = tf.repeat(
            unconditional_context, batch_size, axis=0
        )
        return {"context": encoded_text, "unconditional_context": unconditional_context}

    return serving_fn

In [ ]:
tf.saved_model.save(
    text_encoder,
    "./text_encoder/1/",
    signatures={"serving_default": model_exporter(text_encoder)},
)

In [ ]:
!saved_model_cli show --dir text_encoder/1/ --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['batch_size'] tensor_info:
      dtype: DT_INT32
      shape: ()
      name: serving_default_batch_size:0
  inputs['tokens'] tensor_info:
      dtype: DT_INT32
      shape: ()
      name: serving_default_tokens:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['context'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 77, 768)
      name: StatefulPartitionedCall:0
  outputs['unconditional_context'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 77, 768)
      name: StatefulPartitionedCall:1
Method name is: tensorflow/serving/predict


## Test the encoder SavedModel

In [ ]:
tokenizer = SimpleTokenizer()
prompt = "photograph of an astronaut riding a horse"

phrase = inputs + [49407] * (MAX_PROMPT_LENGTH - len(inputs))
phrase = tf.convert_to_tensor([phrase], dtype=tf.int32)

In [ ]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=text_encoder \
  --model_base_path=/content/text_encoder >server.log 2>&1

In [ ]:
!cat server.log

2022-12-16 04:08:42.282565: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 151781376 exceeds 10% of free system memory.


In [ ]:
!sudo lsof -i -P -n | grep LISTEN

node          7 root   21u  IPv6  18664      0t0  TCP *:8080 (LISTEN)
kernel_ma    33 root    7u  IPv4  18192      0t0  TCP 172.28.0.12:6000 (LISTEN)
colab-fil    46 root    5u  IPv4  18623      0t0  TCP *:3453 (LISTEN)
colab-fil    46 root    6u  IPv6  18624      0t0  TCP *:3453 (LISTEN)
jupyter-n    60 root    6u  IPv4  19505      0t0  TCP 172.28.0.12:9000 (LISTEN)
java       1140 root   19u  IPv4  29517      0t0  TCP 127.0.0.1:33397 (LISTEN)
python3    8234 root    3u  IPv4  44358      0t0  TCP 127.0.0.1:20064 (LISTEN)
python3    8234 root    4u  IPv4  44359      0t0  TCP 127.0.0.1:41843 (LISTEN)
python3    8234 root    9u  IPv4  44541      0t0  TCP 127.0.0.1:36515 (LISTEN)
python3   10742 root   23u  IPv4 742286      0t0  TCP 127.0.0.1:44333 (LISTEN)
python3   10755 root    3u  IPv4 743261      0t0  TCP 127.0.0.1:15408 (LISTEN)
python3   10755 root    4u  IPv4 743262      0t0  TCP 127.0.0.1:33573 (LISTEN)
python3   10755 root    9u  IPv4 743678      0t0  TCP 127.0.0.1:38709 (LISTEN

In [ ]:
import json

data = json.dumps({"signature_name": "serving_default", "instances": [{"tokens": phrase.numpy().tolist(), "batch_size": 1}]})
print("Data: {} ... {}".format(data[:50], data[len(data) - 52 :]))

Data: {"signature_name": "serving_default", "instances": ... 07, 49407, 49407, 49407, 49407]], "batch_size": 1}]}


In [ ]:
import requests

headers = {"content-type": "application/json"}
json_response = requests.post(
    "http://localhost:8501/v1/models/text_encoder:predict", data=data, headers=headers
)

In [ ]:
json_response = json.loads(json_response.text)

In [ ]:
json_response['predictions'][0]['context']

In [ ]:
json_response['predictions'][0]['unconditional_context']

# Diffusion Model in SavedModel

In [ ]:
from tensorflow import keras
from keras_cv.models.stable_diffusion.diffusion_model import DiffusionModel

MAX_PROMPT_LENGTH = 77
IMG_HEIGHT = 512
IMG_WIDTH = 512

diffusion_model = DiffusionModel(IMG_HEIGHT, IMG_WIDTH, MAX_PROMPT_LENGTH)
diffusion_model_weights_fpath = keras.utils.get_file(
    origin="https://huggingface.co/fchollet/stable-diffusion/resolve/main/kcv_diffusion_model.h5",
    file_hash="8799ff9763de13d7f30a683d653018e114ed24a6a819667da4f5ee10f9e805fe",
)
diffusion_model.load_weights(diffusion_model_weights_fpath)

In [ ]:
diffusion_model.inputs

[<KerasTensor: shape=(None, 64, 64, 4) dtype=float32 (created by layer 'input_6')>,
 <KerasTensor: shape=(None, 320) dtype=float32 (created by layer 'input_5')>,
 <KerasTensor: shape=(None, 77, 768) dtype=float32 (created by layer 'input_4')>]

In [ ]:
import math
import tensorflow as tf
from keras_cv.models.stable_diffusion.constants import _ALPHAS_CUMPROD

signature_dict = { "context": tf.TensorSpec(shape=[None, 77, 768], dtype=tf.float32, name="context"),
                   "unconditional_context": tf.TensorSpec(shape=[None, 77, 768], dtype=tf.float32, name="unconditional_context"),
                   "num_steps": tf.TensorSpec(shape=[], dtype=tf.int32, name="num_steps"),
                   "batch_size": tf.TensorSpec(shape=[], dtype=tf.int32, name="batch_size")}

def model_exporter(model: tf.keras.Model):
    IMG_HEIGHT = 512
    IMG_WIDTH = 512
    MAX_PROMPT_LENGTH = 77

    SEED = None

    @tf.function
    def _get_timestep_embedding(timestep, batch_size, dim=320, max_period=10000):
        half = dim // 2
        freqs = tf.math.exp(
            -math.log(max_period) * tf.range(0, half, dtype=tf.float32) / half
        )
        args = tf.convert_to_tensor([timestep], dtype=tf.float32) * freqs
        embedding = tf.concat([tf.math.cos(args), tf.math.sin(args)], 0)
        embedding = tf.reshape(embedding, [1, -1])
        return tf.repeat(embedding, batch_size, axis=0)

    @tf.function(input_signature=[signature_dict])
    def serving_fn(string_input):
        img_height = round(IMG_HEIGHT / 128) * 128
        img_width = round(IMG_WIDTH / 128) * 128
        unconditional_guidance_scale = 7.5

        batch_size = string_input["batch_size"]
        num_steps = string_input["num_steps"]

        context = string_input["context"]
        print(context)
        unconditional_context = string_input["unconditional_context"]
        print(unconditional_context)

        latent = tf.random.normal(
            (batch_size, img_height // 8, img_width // 8, 4), seed=SEED
        )

        timesteps = tf.range(1, 1000, 1000 // num_steps)
        _ALPHAS_CUMPROD_t = tf.convert_to_tensor(_ALPHAS_CUMPROD)
        alphas = []

        for t in timesteps:
          alphas.append(_ALPHAS_CUMPROD_t[t])
        alphas_prev = [1.0] + alphas[:-1]

        print(alphas)

        index = 0
        for t in timesteps:
          latent_prev = latent
          t_emb = _get_timestep_embedding(t, batch_size)
          unconditional_latent = model(
              [latent, t_emb, unconditional_context]
          )
          latent = model([latent, t_emb, context])
          latent = unconditional_latent + unconditional_guidance_scale * (
              latent - unconditional_latent
          )
          a_t, a_prev = alphas[index], alphas_prev[index]

          index = index + 1


        # for index, timestep in enumerate(timesteps)[::-1]:
            # latent_prev = latent  # Set aside the previous latent vector
            # t_emb = _get_timestep_embedding(timestep, batch_size)
            # unconditional_latent = model(
            #     [latent, t_emb, unconditional_context]
            # )
            # latent = model([latent, t_emb, context])
            # latent = unconditional_latent + unconditional_guidance_scale * (
            #     latent - unconditional_latent
            # )
            # a_t, a_prev = alphas[index], alphas_prev[index]
            # pred_x0 = (latent_prev - math.sqrt(1 - a_t) * latent) / math.sqrt(a_t)
            # latent = latent * math.sqrt(1.0 - a_prev) + math.sqrt(a_prev) * pred_x0

        return {"encoded_text": timesteps}

    return serving_fn

In [ ]:
tf.saved_model.save(
    diffusion_model,
    "./diffusion_model/1/",
    signatures={"serving_default": model_exporter(diffusion_model)},
)

Tensor("context:0", shape=(None, 77, 768), dtype=float32)
Tensor("unconditional_context:0", shape=(None, 77, 768), dtype=float32)
[<tf.Tensor 'while/strided_slice:0' shape=() dtype=float32>]


TypeError: ignored